In [1]:
import statsmodels.formula.api as smf
import numpy as np
import statsmodels.api as sm
from tqdm import tnrange, tqdm_notebook
import pandas as pd
import itertools
import re
from sklearn.model_selection import train_test_split
indeed = pd.read_csv('indeed_Lin_Reg.csv')

### Below function is to help label data as Data Science related or not, which is our response variable

In [2]:
def categorize(s):
    """categorizes the job position to bins of SEAM (scientist, data engineer, analyst, ML)"""
    
    patterns = ['.*data sci.*','.*data.*eng.*', '.*analy.*', '.*machine.*']
    for idx, pattern in enumerate(patterns):
        reg = re.compile(pattern)
        if reg.search(s):
            return idx
    return -1

### Don't want missing data, so drop those rows

In [3]:
indeed = indeed[~indeed.position.isna()]

### Normalize Text and label data (using one hot encoding)

In [4]:
indeed['position'] = indeed['position'].apply(lambda x : x.lower())
        
indeed['label'] = -1
for idx, row in indeed.iterrows():
    cat = categorize(row.position)
    if cat != -1:
        indeed.loc[idx, 'label'] = 'DS'
    else:
        indeed.loc[idx, 'label'] = 'Other'

In [5]:
indeed, test = train_test_split(indeed, test_size=0.25, random_state=42)

### In order to make the formula, have to change c# to c_sharp and scikit-learn to scikit_learn

In [6]:
indx = list(indeed.columns).index('c#')
indeed.columns = list(indeed.columns)[:indx] + ['c_sharp'] + list(indeed.columns)[indx+1:]
indx = list(indeed.columns).index('scikit-learn')
indeed.columns = list(indeed.columns)[:indx] + ['scikit_learn'] + list(indeed.columns)[indx+1:]

In [7]:
formula = 'label ~ ' + ' + '.join(list(indeed.columns)[5:42])
print(formula)

label ~ python + r + sql + spark + hadoop + java + sas + tableau + hive + scala + c++ + aws + tensorflow + matlab + c + excel + linux + nosql + azure + scikit_learn + spss + pandas + numpy + pig + d3 + keras + javascript + c_sharp + perl + hbase + docker + git + mysql + mongodb + cassandra + pytorch + caffe


In [8]:
model = smf.glm(formula, data=indeed, family=sm.families.Binomial()).fit()

In [9]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Generalized Linear Model Regression Results                       
=========================================================================================
Dep. Variable:     ['label[DS]', 'label[Other]']   No. Observations:                 5214
Model:                                       GLM   Df Residuals:                     5177
Model Family:                           Binomial   Df Model:                           36
Link Function:                             logit   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -2833.0
Date:                           Fri, 11 Oct 2019   Deviance:                       5666.0
Time:                                   19:29:57   Pearson chi2:                 5.39e+03
No. Iterations:                                5   Covariance Type:             nonrobust
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.1686      0.049    -23.863      0.000      -1.265      -1.073
python           1.0943      0.089     12.306      0.000       0.920       1.269
r                0.2770      0.083      3.343      0.001       0.115       0.439
sql              1.0346      0.095     10.928      0.000       0.849       1.220
spark            0.4800      0.149      3.215      0.001       0.187       0.773
hadoop           0.4803      0.154      3.128      0.002       0.179       0.781
java            -0.0399      0.115     -0.346      0.729      -0.266       0.186
sas              0.5055      0.134      3.777      0.000       0.243       0.768
tableau          0.7742      0.166      4.672      0.000       0.449       1.099
hive             0.2589      0.206      1.255      0.209      -0.145       0.663
scala            0.8433      0.175      4.828      0.000       0.501       1.186
c               -0.1030      0.118     -0.876      0.381      -0.333       0.127
aws              0.0874      0.136      0.645      0.519      -0.178       0.353
tensorflow       1.0283      0.218      4.711      0.000       0.600       1.456
matlab          -0.1733      0.129     -1.347      0.178      -0.425       0.079
excel            0.3895      0.094      4.130      0.000       0.205       0.574
linux           -0.3581      0.132     -2.704      0.007      -0.618      -0.099
nosql           -0.3927      0.197     -1.995      0.046      -0.779      -0.007
azure           -0.8080      0.211     -3.836      0.000      -1.221      -0.395
scikit_learn     0.1994      0.288      0.692      0.489      -0.365       0.764
spss             0.5977      0.213      2.805      0.005       0.180       1.015
pandas           1.0421      0.357      2.916      0.004       0.342       1.742
numpy           -0.1173      0.373     -0.315      0.753      -0.849       0.614
pig              1.1821      0.346      3.419      0.001       0.504       1.860
d3               0.5832      0.336      1.736      0.083      -0.075       1.242
keras            0.9288      0.363      2.559      0.011       0.217       1.640
javascript      -0.6604      0.166     -3.980      0.000      -0.986      -0.335
c_sharp         -0.3356      0.189     -1.775      0.076      -0.706       0.035
perl            -0.8921      0.166     -5.376      0.000      -1.217      -0.567
hbase            0.0494      0.362      0.137      0.891      -0.659       0.758
docker          -0.7522      0.246     -3.058      0.002      -1.234      -0.270
git             -0.0924      0.206     -0.448      0.654      -0.497       0.312
mysql            0.0138      0.246      0.056      0.955      -0.468       0.496
mongodb         -0.1950      0.306     -0.638      0.524      -0.794       0.404
cassandra        0.1250      0.353      0.354      0.72

### From the Wald's Test we see in the above summary table, we see 14 predictors are insignificant, we'll start by dropping these predictors and then will do best subset on the remaining predictors

In [10]:
pvals = model.pvalues
pvals = pvals.reset_index(drop=False)
pvals.columns = ['skill', 'pval']
preds = []
for coefficient in pvals.iterrows():
    
    if coefficient[1][1] > 0.05 and coefficient[1][0] != 'Intercept':
        indeed.drop(coefficient[1][0], axis=1, inplace=True)
    else:
        preds.append(coefficient[1][0])

In [11]:
preds.remove('Intercept')

In [12]:
preds

['python',
 'r',
 'sql',
 'spark',
 'hadoop',
 'sas',
 'tableau',
 'scala',
 'tensorflow',
 'excel',
 'linux',
 'nosql',
 'azure',
 'spss',
 'pandas',
 'pig',
 'keras',
 'javascript',
 'perl',
 'docker',
 'pytorch']

### Trying to implement stepwise selection. Best subset takes too long due to large amount of predictors.

In [13]:
#Initialization variables
Y = indeed.label #credit.Balance
X = indeed.drop(columns='label', axis=1) #credit.drop(columns = 'Balance', axis = 1)
k = X.shape[1]

remaining_features = preds
features = []
features_list = dict()
alpha = 0.15

for i in tnrange(0,k, desc='Loop...'):
    #Variables to add
    best_pval = np.inf
    best_candidate = None
    best_feature = None
    
    for combo in itertools.combinations(remaining_features,1):
        formula = 'label ~ ' + ' + '.join(features + [combo[0]])
        candidate = smf.glm(formula, data=indeed, family=sm.families.Binomial()).fit()  #Store temp result 

        pvals = candidate.pvalues.reset_index(drop=False)
        pvals.columns = ['skill', 'pval']
        pval = pvals.loc[pvals.shape[0]-1, 'pval']
        skill = pvals.loc[pvals.shape[0]-1, 'skill']
        if pval < alpha and pval < best_pval:
            best_pval = pval
            best_candidate = candidate
            best_feature = skill

    #Check for features to remove
    if best_candidate is None: continue
    pvals = best_candidate.pvalues.reset_index(drop=False)
    pvals.columns = ['skill', 'pval']
    for coefficient in pvals.iterrows():
        if coefficient[1][1] > alpha and coefficient[1][0] != 'Intercept':
            features.remove(coefficient[1][0])
    
    #Updating variables for next loop
    features.append(best_feature)
    remaining_features.remove(best_feature)
    

In [14]:
features

['python',
 'sql',
 'spark',
 'sas',
 'perl',
 'javascript',
 'tableau',
 'tensorflow',
 'scala',
 'pig',
 'excel',
 'docker',
 'pandas',
 'r',
 'azure',
 'hadoop',
 'linux',
 'spss',
 'pytorch',
 'keras',
 'nosql']

In [15]:
formula = 'label ~ ' + ' + '.join(features)

In [16]:
formula

'label ~ python + sql + spark + sas + perl + javascript + tableau + tensorflow + scala + pig + excel + docker + pandas + r + azure + hadoop + linux + spss + pytorch + keras + nosql'

In [17]:
model = smf.glm(formula, data=indeed, family=sm.families.Binomial()).fit()

In [18]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Generalized Linear Model Regression Results                       
=========================================================================================
Dep. Variable:     ['label[DS]', 'label[Other]']   No. Observations:                 5214
Model:                                       GLM   Df Residuals:                     5192
Model Family:                           Binomial   Df Model:                           21
Link Function:                             logit   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -2841.0
Date:                           Fri, 11 Oct 2019   Deviance:                       5682.0
Time:                                   19:30:07   Pearson chi2:                 5.42e+03
No. Iterations:                                5   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.1828      0.048    -24.563      0.000      -1.277      -1.088
python         1.0498      0.083     12.675      0.000       0.887       1.212
sql            1.0545      0.093     11.344      0.000       0.872       1.237
spark          0.5313      0.143      3.704      0.000       0.250       0.812
sas            0.4973      0.133      3.736      0.000       0.236       0.758
perl          -0.9498      0.162     -5.877      0.000      -1.267      -0.633
javascript    -0.7018      0.156     -4.499      0.000      -1.007      -0.396
tableau        0.8330      0.163      5.097      0.000       0.513       1.153
tensorflow     0.9699      0.193      5.020      0.000       0.591       1.349
scala          0.8517      0.166      5.146      0.000       0.527       1.176
pig            1.3020      0.320      4.073      0.000       0.676       1.929
excel          0.3837      0.094      4.076      0.000       0.199       0.568
docker        -0.6966      0.235     -2.969      0.003      -1.156      -0.237
pandas         1.0528      0.283      3.722      0.000       0.498       1.607
r              0.2731      0.081      3.377      0.001       0.115       0.432
azure         -0.7741      0.199     -3.890      0.000      -1.164      -0.384
hadoop         0.5072      0.146      3.473      0.001       0.221       0.793
linux         -0.3674      0.128     -2.866      0.004      -0.619      -0.116
spss           0.5967      0.213      2.804      0.005       0.180       1.014
pytorch       -0.8544      0.306     -2.790      0.005      -1.454      -0.254
keras          0.9677      0.361      2.684      0.007       0.261       1.675
nosql         -0.3659      0.188     -1.950      0.051      -0.734       0.002
==============================================================================
"""

### Find Mis-classification Error rate with a threshold of 0.5

In [19]:
test['prediction_prob'] = model.predict(test)
test.loc[test.prediction_prob > 0.5, 'prediction'] = 'DS'
test.prediction.fillna('Other', inplace=True)

In [20]:
test['flag'] = test.label != test.prediction

In [21]:
test.flag.sum() / test.shape[0]

0.2553191489361702

### Find Mis-Classification error rates at other thresholds

In [22]:
def seq(start, stop, step):
    while start <= stop:
        yield start
        start = round(start+step, 2)
    
s = seq(0.1, 0.9, 0.01)

threshold = []
misclass_error = []

for i in s:
    if i == 0.91:
        s.close()
    threshold.append(i)
    test.loc[test.prediction_prob > i, 'prediction'] = 'DS'
    test.loc[test.prediction_prob < i, 'prediction'] = 'Other'    
    test['flag'] = test.label != test.prediction
    misclass_error.append(test.flag.sum() / test.shape[0])


### Want to check for Multicollinearity, so calculated Efron's Pseudo $R^2$ and VIF

In [23]:
R2 = []
X = indeed[features]
for feature in features:
    x_feature = features.copy()
    x_feature.remove(feature)
    formula = feature + ' ~ ' + ' + '.join(x_feature)
    m = smf.glm(formula, data=X, family=sm.families.Binomial()).fit()
    predictions = m.predict(X)
    numerator = X[feature] - predictions
    numerator = numerator.to_numpy() @ numerator.to_numpy()
    denominator = X[feature] - X[feature].mean()
    denominator = denominator.to_numpy() @ denominator.to_numpy()
    R2.append(1 - (numerator / denominator))

In [24]:
R2

[0.46437742750065114,
 0.3256285579606303,
 0.5063407106267137,
 0.24198736339014182,
 0.10784873181319099,
 0.05090654334319655,
 0.19055778590768724,
 0.39968215384800565,
 0.24090886071593554,
 0.15654210547553593,
 0.09387088028439516,
 0.09504872833004963,
 0.09117332347894935,
 0.3037216022612116,
 0.051790808860393334,
 0.4866136206974736,
 0.09710186201454107,
 0.2032831228431463,
 0.2734825061577294,
 0.3252485205369269,
 0.18122372702200507]

### None of the VIF values are greater than 4, so no suspection of multicollinearity

In [25]:
VIF = list(map(lambda x: 1/(1-x), R2))

In [26]:
VIF

[1.8669862909879804,
 1.4828623183922134,
 2.0256886105992797,
 1.3192392206974386,
 1.1208861497584155,
 1.0536370185529629,
 1.2354186408740415,
 1.6657842281550468,
 1.317364869972199,
 1.1855956373065821,
 1.103595479101099,
 1.1050318744286105,
 1.1003198143654376,
 1.436207131009045,
 1.0546196022400378,
 1.9478506643642832,
 1.1075446475404127,
 1.255151018726474,
 1.3764293475046085,
 1.4820271321164649,
 1.221334854224428]